In [1]:
import os
import shutil
import numpy as np
from ase.build import bulk
from pylammpsmpi import LammpsASELibrary
from pyiron_atomistics.lammps.lammps import lammps_function

/Users/janssen/mambaforge/lib/python3.12/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
structure = bulk("Al", cubic=True).repeat([2,2,2])
potential = '2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1'
md_parameter = {"temperature": 500.0, "pressure": 0.0, "n_ionic_steps": 1000, "n_print": 100}

# File based convergence loop

In [3]:
shell_output, parsed_output, job_crashed = lammps_function(
    working_directory=os.path.abspath("lmp"), 
    structure=structure, 
    potential=potential, 
    calc_mode="md", 
    calc_kwargs=md_parameter, 
    cutoff_radius=None, 
    units="metal", 
    bonds_kwargs={}, 
    enable_h5md=False,
    write_restart_file=True
)

In [4]:
parsed_output["generic"]["temperature"], np.std(parsed_output["generic"]["temperature"])

(array([562.404207  , 486.25918764, 389.49038218, 373.19022618,
        510.47420031, 545.08297132, 592.08950286, 469.19345301,
        471.73704907, 488.15950964, 422.78586373]),
 65.77990564783019)

In [5]:
while np.std(parsed_output["generic"]["temperature"]) > 50:
    shutil.copy("lmp/restart.out", "restart.out")
    shutil.rmtree("lmp")
    shell_output, parsed_output, job_crashed = lammps_function(
        working_directory=os.path.abspath("lmp"), 
        structure=structure, 
        potential=potential, 
        calc_mode="md", 
        calc_kwargs=md_parameter, 
        cutoff_radius=None, 
        units="metal", 
        bonds_kwargs={}, 
        enable_h5md=False,
        write_restart_file=True,
        read_restart_file=True,
        restart_file=os.path.abspath("restart.out"),
    )
    print(parsed_output["generic"]["temperature"], np.std(parsed_output["generic"]["temperature"]))

[422.78586373 373.26161557 391.97528516 641.4276823  644.84452375
 477.75737506 350.57497329 394.50161946 508.70080264 638.08597554
 410.54102853] 108.83410654535234
[410.54102853 510.29162697 472.11008241 474.55780499 485.70602676
 544.5581859  539.32292264 542.60816133 468.62245098 599.92248045
 455.61717725] 50.473224024678316
[455.61717725 568.11403744 480.42383707 534.64601625 569.52111907
 451.18417071 431.61227141 452.63254984 514.01967062 590.95588602
 530.02504922] 53.20362391310318
[530.02504922 406.58330785 551.63542307 483.08075438 416.255085
 374.57224753 468.09430964 646.16327963 546.96735772 483.11098791
 437.41058746] 75.01577311038145
[437.41058746 349.53010048 358.58525639 452.37593825 473.43731953
 507.93251316 540.59423333 499.35630192 448.5342581  377.58252091
 557.97174714] 67.29163720475582
[557.97174714 508.33790086 402.26417865 543.75905758 538.64149611
 470.78547211 355.35394009 344.91374651 596.90248155 559.26685534
 574.0330973 ] 85.4902284350088
[574.033097

# Lammps Library 

In [6]:
shutil.copy(os.path.join(os.environ["CONDA_PREFIX"], "share/iprpy/potential_LAMMPS/2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1/Al-Mg.eam.fs"), "Al-Mg.eam.fs")

'Al-Mg.eam.fs'

In [7]:
lmp = LammpsASELibrary()

In [8]:
lmp.interactive_structure_setter(structure=structure, units="metal", dimension="3", boundary="p p p", atom_style="atomic", el_eam_lst=["Al", "Mg"])

In [9]:
lmp_str = """\
pair_style eam/fs
pair_coeff * * Al-Mg.eam.fs Al Mg
fix ensemble all npt temp 500.0 500.0 0.1 iso 0.0 0.0 1.0
variable dumptime  equal 100 
variable thermotime  equal 100 
timestep 0.001
velocity all create 1000.0 80996 dist gaussian
dump 1 all custom ${dumptime} dump.out id type xsu ysu zsu fx fy fz vx vy vz
dump_modify 1 sort id format line "%d %d %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g"
thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
thermo ${thermotime}"""

In [10]:
for l in lmp_str.split("\n"):
    lmp.interactive_lib_command(l)

In [11]:
temperature_lst = [1000.0, 0.0]

In [12]:
while np.std(temperature_lst) > 50.0:
    temperature_lst = []
    for i in range(10):
        lmp.interactive_lib_command("run 100")
        temperature_lst.append(lmp.interactive_temperatures_getter())
    print(temperature_lst, np.std(temperature_lst))

[507.46684483912094, 366.3649499477118, 290.61690982724946, 445.78568812327956, 636.2673040684064, 536.9144916639995, 462.7074972832532, 518.8638005735679, 495.1737716726091, 537.1636963552056] 91.47092292889148
[532.3207920398262, 508.63308288504714, 513.6169841087938, 424.54044144150504, 552.0492949598025, 611.601959420703, 611.5802474562507, 497.6292778475882, 517.0685237478514, 428.8586245172485] 60.045523311835375
[484.09080574821854, 448.138025296258, 576.1868762978492, 441.38298188068745, 441.1966264175477, 521.791791322101, 597.5457206556606, 548.6115497432645, 435.44234588579326, 384.004680346689] 66.43633864816368
[480.17028420111717, 545.2592345793655, 515.3382156006019, 578.2362052201769, 445.37975286409784, 461.10236884562727, 444.95809001806106, 507.0037220012684, 457.3172170197647, 507.9161033880662] 42.34239222344232


In [13]:
lmp.close()